In [24]:
import numpy as np
import pandas as pd

x_train = pd.read_csv(r'C:\Users\flore\OneDrive\Bureau\X_train_Hi5.csv')

C:\Users\flore\AppData\Local\Temp\ipykernel_22964\3224182190.py:4: DtypeWarning: Columns (108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  x_train = pd.read_csv(r'C:\Users\flore\OneDrive\Bureau\X_train_Hi5.csv')


In [ ]:
def fill_missing_values_prelev(x_train):
    prelev = x_train.iloc[:, [4]+[11]+[12]+[15]+list(range(108, 130)) + [135]]
    prelev['piezo_measurement_date'] = pd.to_datetime(prelev['piezo_measurement_date'])
    prelev['year'] = pd.to_datetime(prelev['piezo_measurement_date']).dt.year
    prelev['month'] = pd.to_datetime(prelev['piezo_measurement_date']).dt.month
    prelev['day'] = pd.to_datetime(prelev['piezo_measurement_date']).dt.day

    # Custom function to select row with the fewest NaNs
    def row_with_fewest_nans(group):
        # Count NaNs in each row
        nan_counts = group.isna().sum(axis=1)
        # Get the index of the row with the fewest NaNs
        min_nan_row_idx = nan_counts.idxmin()
        # Return the row with the fewest NaNs
        return group.loc[min_nan_row_idx]

    # Group by piezo_station_latitude, piezo_station_longitude, and year, and apply the custom function
    reduced_data = prelev.groupby(['piezo_station_latitude', 'piezo_station_longitude', 'year'], as_index=False).apply(row_with_fewest_nans)

    # Drop the specified columns
    reduced_data = reduced_data.drop(['piezo_station_department_name', 
                                    'piezo_measurement_date', 
                                    'month', 
                                    'day','piezo_groundwater_level_category'], axis=1)

    from scipy.spatial.distance import cdist

    def fill_missing_values_with_nearest(df):
        # Helper function to calculate geospatial distance
        def get_nearest_station_values(row, candidates):
            lat, lon = row['piezo_station_latitude'], row['piezo_station_longitude']
            distances = cdist([(lat, lon)], candidates[['piezo_station_latitude', 'piezo_station_longitude']])
            nearest_idx = distances.argmin()
            return candidates.iloc[nearest_idx]

        # Exclude non-relevant columns from the fully NaN check
        columns_to_check = [col for col in df.columns if col not in ['piezo_station_latitude', 'piezo_station_longitude', 'year', 'prelev_other_volume_sum']]
        
        # Step 1: Fill NaNs with mode/median within the same station group
        grouped = df.groupby(['piezo_station_latitude', 'piezo_station_longitude'])
        
        for column in df.select_dtypes(include=['object']).columns:
            df[column] = grouped[column].transform(
                lambda x: x.mode()[0] if not x.mode().empty else np.nan)
        
        for column in df.select_dtypes(include=['float64', 'int64']).columns:
            df[column] = grouped[column].transform('median')

        # Step 2: Handle rows where all columns in `columns_to_check` are NaN
        fully_nan_rows = df[columns_to_check].isna().all(axis=1)
        
        # Find stations that have no NaNs
        valid_stations = df.dropna(subset=columns_to_check).drop_duplicates(subset=['piezo_station_latitude', 'piezo_station_longitude'])

        # Fill in missing rows with values from the nearest valid station
        for idx in df[fully_nan_rows].index:
            row = df.loc[idx]
            if not valid_stations.empty:
                nearest_station_values = get_nearest_station_values(row, valid_stations)
                for col in columns_to_check:
                    df.at[idx, col] = nearest_station_values[col]

        
        return df

        # Apply the fill function
    reduced = fill_missing_values_with_nearest(reduced_data)
    reduced['prelev_other_volume_sum'].fillna(0, inplace=True)
    rows_with_nans = reduced[reduced.isna().any(axis=1)]
        # Define the target station's latitude and longitude
    target_lat = rows_with_nans.iloc[0]['piezo_station_latitude']
    target_lon = rows_with_nans.iloc[0]['piezo_station_longitude']

    # Precision level (number of decimal places to round to)
    precision = 5  # Adjust this based on your needs

    # Haversine formula to calculate the distance between two lat/lon points
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371  # Earth radius in kilometers
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        return R * c

    # Round latitude and longitude to the specified precision
    reduced['lat_lon_rounded'] = reduced.apply(
        lambda row: (round(row['piezo_station_latitude'], precision), round(row['piezo_station_longitude'], precision)),
        axis=1
    )

    # Define the rounded target station's latitude and longitude
    target_lat_rounded = round(target_lat, precision)
    target_lon_rounded = round(target_lon, precision)

    # Calculate distances to all stations with the same precision rounding
    reduced['distance'] = reduced.apply(
        lambda row: haversine(target_lat, target_lon, row['piezo_station_latitude'], row['piezo_station_longitude']),
        axis=1
    )

    # Exclude the target station itself, based on the rounded lat/lon values
    closest_station = reduced[
        (reduced['lat_lon_rounded'] != (target_lat_rounded, target_lon_rounded))
    ].sort_values(by='distance').iloc[0]

    # Show the lines corresponding to the closest station
    closest_lat = closest_station['piezo_station_latitude']
    closest_lon = closest_station['piezo_station_longitude']

    closest_lines = reduced[
        (reduced['piezo_station_latitude'] == closest_lat) &
        (reduced['piezo_station_longitude'] == closest_lon)
    ]

    # Iterate over rows_with_nans to fill NaNs
    for idx, row in rows_with_nans.iterrows():
        year = row['year']  # Get the year of the current row
        matching_row = closest_lines[closest_lines['year'] == year]  # Find matching year in closest_lines

        if not matching_row.empty:
            # Fill NaN values with corresponding values from matching_row
            for column in rows_with_nans.columns:
                if pd.isna(row[column]):  # Check if the value is NaN
                    rows_with_nans.at[idx, column] = matching_row.iloc[0][column]

    # Update the original reduced dataset with the filled rows
    reduced.update(rows_with_nans)
    # Function to fill missing values based on the matching latitude, longitude, and year in 'reduced'
    def fill_missing_with_closest(df_prelev, df_reduced, precision=5):
        # Round the latitudes and longitudes in both datasets for comparison with precision
        df_prelev['lat_lon_rounded'] = df_prelev.apply(
            lambda row: (round(row['piezo_station_latitude'], precision), round(row['piezo_station_longitude'], precision)),
            axis=1
        )
        
        df_reduced['lat_lon_rounded'] = df_reduced.apply(
            lambda row: (round(row['piezo_station_latitude'], precision), round(row['piezo_station_longitude'], precision)),
            axis=1
        )

        # Loop through each row in 'df_prelev' where there are missing values
        for idx, row in df_prelev[df_prelev.isna().any(axis=1)].iterrows():
            # Filter out rows in df_reduced that match the same lat, lon, and year
            matching_reduced = df_reduced[
                (df_reduced['lat_lon_rounded'] == row['lat_lon_rounded']) & 
                (df_reduced['year'] == row['year'])
            ]
            
            if not matching_reduced.empty:
                # For each column with NaN in df_prelev, replace it with the corresponding value from df_reduced
                for col in row.index:
                    if pd.isna(row[col]) and col in matching_reduced.columns:
                        df_prelev.at[idx, col] = matching_reduced.iloc[0][col]

        return df_prelev

    # Example usage to fill missing values in 'prelev' using 'reduced'
    df_prelev_filled = fill_missing_with_closest(prelev.copy(), reduced.copy())
    # Ensure that the columns in both DataFrames are aligned
    common_columns = [col for col in x_train.columns if col in df_prelev_filled.columns]

    # Replace the columns in x_train with those from df_prelev_filled
    x_train[common_columns] = df_prelev_filled[common_columns]

    return x_train
x_train_filled = fill_missing_values_prelev(x_train)

C:\Users\flore\AppData\Local\Temp\ipykernel_22964\1051602779.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prelev['piezo_measurement_date'] = pd.to_datetime(prelev['piezo_measurement_date'])
C:\Users\flore\AppData\Local\Temp\ipykernel_22964\1051602779.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prelev['year'] = pd.to_datetime(prelev['piezo_measurement_date']).dt.year
C:\Users\flore\AppData\Local\Temp\ipykernel_22964\1051602779.py:5: SettingWithCopyWarning: 
A value is trying to be set on a cop

       piezo_station_longitude  piezo_station_latitude  \
0                     9.182972               41.399732   
1                     9.182972               41.399732   
2                     9.182972               41.399732   
3                     9.182972               41.399732   
4                     9.184641               41.404117   
...                        ...                     ...   
10828                 1.818080               50.867038   
10829                 2.169599               50.971087   
10830                 2.169599               50.971087   
10831                 2.169599               50.971087   
10832                 2.169599               50.971087   

      prelev_structure_code_0  prelev_volume_0 prelev_usage_label_0  \
0                         NaN              NaN                  NaN   
1                         NaN              NaN                  NaN   
2                         NaN              NaN                  NaN   
3                  